# 1、Working with data

## torch.utils.data.Dataset
存储数据样本与相对应的标签
## torch.utils.data.DataLoader
给Dataset包装了一个可迭代对象

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

## PyTorch 提供特定领域的库，例如 TorchText、TorchVision 和 TorchAudio，所有这些库都包含数据集。

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

## 在这里，我们定义了 64 的批大小，即 dataloader 迭代中的每个元素将返回一批 64 个特征和标签。

In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


# 2、Creating Models

In [4]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


# 3、Optimizing the Model Parameters

## 训练模型需要损失函数和优化器

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

## 在单个训练循环中，模型对训练数据集进行预测（分批输入），并反向传播预测误差以调整模型的参数。

In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

## 根据测试数据集检查模型的性能，以确保它正在学习

In [7]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

## 训练过程在多次迭代（epochs）中进行。在每个时期，模型都会学习参数以做出更好的预测。我们在每个时期打印模型的准确性和损失；我们希望看到每个 epoch 的准确率增加和损失减少。

In [8]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.298091  [    0/60000]
loss: 2.294713  [ 6400/60000]
loss: 2.275717  [12800/60000]
loss: 2.278152  [19200/60000]
loss: 2.262997  [25600/60000]
loss: 2.230045  [32000/60000]
loss: 2.239455  [38400/60000]
loss: 2.200585  [44800/60000]
loss: 2.205815  [51200/60000]
loss: 2.192594  [57600/60000]
Test Error: 
 Accuracy: 42.6%, Avg loss: 2.174325 

Epoch 2
-------------------------------
loss: 2.180664  [    0/60000]
loss: 2.171505  [ 6400/60000]
loss: 2.119699  [12800/60000]
loss: 2.141972  [19200/60000]
loss: 2.095224  [25600/60000]
loss: 2.033992  [32000/60000]
loss: 2.068475  [38400/60000]
loss: 1.985889  [44800/60000]
loss: 1.999347  [51200/60000]
loss: 1.942659  [57600/60000]
Test Error: 
 Accuracy: 58.5%, Avg loss: 1.927404 

Epoch 3
-------------------------------
loss: 1.955478  [    0/60000]
loss: 1.927062  [ 6400/60000]
loss: 1.814618  [12800/60000]
loss: 1.864793  [19200/60000]
loss: 1.754169  [25600/60000]
loss: 1.695737  [32000/600

# Saving Models

## 保存模型的常用方法是序列化内部状态字典（包含模型参数）。

In [9]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


# Loading Models

## 加载模型的过程包括重新创建模型结构并将状态字典加载到其中。

In [10]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [11]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
